# Fourier transform

In [ ]:
using Revise
using Plots
using LinearAlgebra
using MultiScales

newaxis = [CartesianIndex()]

In [ ]:
using ITensors

println(ITensors.blas_get_num_threads())

In [ ]:
function eval(psi, sites, el)
    V = ITensor(1.)
    for j=1:length(el)
        V *= (psi[j] * state(sites[j], el[j]))
    end
    return scalar(V)
end

In [ ]:
nbit = 2
N = 2^nbit

sites = siteinds("Qubit", nbit)

In [ ]:
Z = [op("Z", sites, n) for n in 1:nbit]
;

In [ ]:
mps = MPS(sites, ["1" for n in 1:nbit])

In [ ]:
M = MPO(sites, ["Z" for n in 1:nbit])

In [ ]:
os = Prod{Op}()
os *= Op("Z", 1)
M = MPO(os, sites)

In [ ]:
M = MPO(Op("Z", 1), sites)

In [ ]:
# Control: 1
# Target: 2
cnot = op("CNOT", sites, 1, 2)

In [ ]:
state0 = MPS(sites, ["1" for n in 1:nbit])
for i2 in 1:2, i1 in 1:2
    @show i1, i2, eval(state0, sites, [i1, i2])
end
res = noprime(contract(MPO(cnot, sites), state0))

In [ ]:
for i2 in 1:2, i1 in 1:2
    @show i1, i2, eval(res, sites, [i1, i2])
end